Q
====

http://mnemstudio.org/path-finding-q-learning-tutorial.htm

https://gist.github.com/kastnerkyle/d127197dcfdd8fb888c2

文章里有些小细节的数字有错误。给出的代码比文章描述的过程要稍微复杂。

这个是房间的地图。有门的房间表示之间可以互通。

![q_room.gif](q_room.gif)

这个是状态-动作转移图。状态5是结束态。

![q_model.gif](q_model.gif)

这个是$Q*$最优Q函数。红线代表$\pi$最优策略。

![q_converge.gif](q_converge.gif)


Q的更新策略只有一条

    Q(state, action) = R(state, action) + Gamma * Max[Q(next state, all actions)]
    
$$
Q(s, a) = R(s, a) + \gamma * \max{Q(s', a')}
$$


下面的R代表了从一个状态a到另一个状态b的reward。-1表示a到b不通。

In [1]:
import numpy as np

n_states = 6
n_actions = 6
goal = set([5])
gamma = .8

R = np.array([
    [-1, -1, -1, -1, 0, -1],
    [-1, -1, -1,  0, -1, 100],
    [-1, -1, -1,  0, -1, -1],
    [-1,  0,  0, -1,  0, -1],
    [ 0, -1, -1,  0, -1, 100],
    [-1,  0, -1, -1,  0, 100]
    ], dtype=np.float32)

Q(s, a)行代表状态，列代表动作。这个问题里两个数都是6。Q被初始化为0。

In [2]:
Q = np.zeros_like(R)

def update(s, a, s1):
    Q[s][a] = R[s][a] + gamma * max(Q[s1])

按照文章中的例子，可以试一下，假设当前在状态s=1，动作为a=5，下一个状态s1=5。更新一次Q的结果如下。

In [3]:
update(1, 5, 5)
print(Q)

[[   0.    0.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0.  100.]
 [   0.    0.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0.    0.]]


再走一步，当前状态s=3，动作为a=1，下一个状态为s1=1。再更新一次的结果：

In [4]:
update(3, 1, 1)
print(Q)

[[   0.    0.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0.  100.]
 [   0.    0.    0.    0.    0.    0.]
 [   0.   80.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0.    0.]]


使用这样简单的策略不断更新。在episode开始的时候随机选择一个状态。在每个状态随机选择一个动作进行更新。

In [5]:
for _ in range(2000):
    s = np.random.randint(0, n_states)
    while 1:
        a = s1 = np.random.choice(np.flatnonzero(R[s] != -1))
        update(s, a, s1)
        if s1 in goal:
            break
        s = s1

np.set_printoptions(precision=3)
print(Q)

[[   0.    0.    0.    0.  400.    0.]
 [   0.    0.    0.  320.    0.  500.]
 [   0.    0.    0.  320.    0.    0.]
 [   0.  400.  256.    0.  400.    0.]
 [ 320.    0.    0.  320.    0.  500.]
 [   0.  400.    0.    0.  400.  500.]]


In [6]:
print(Q/5)

[[  0.    0.    0.    0.   80.    0. ]
 [  0.    0.    0.   64.    0.  100. ]
 [  0.    0.    0.   64.    0.    0. ]
 [  0.   80.   51.2   0.   80.    0. ]
 [ 64.    0.    0.   64.    0.  100. ]
 [  0.   80.    0.    0.   80.  100. ]]


使用epsilon greedy。

In [7]:
epsilon = .05

Q = np.zeros((n_states, n_actions))

for _ in range(2000):
    s = np.random.randint(0, n_states)
    while 1:
        if np.random.random() > epsilon and np.sum(Q[s]) > 0:
            a = np.argmax(Q[s])
        else:
            a = np.random.choice(np.flatnonzero(R[s] != -1))
        s1 = a
        Q[s][a] = R[s][a] + gamma * max(Q[s1])
        if s1 in goal:
            break
        s = s1

print(Q/5)

[[   0.     0.     0.     0.    80.     0. ]
 [   0.     0.     0.    64.     0.   100. ]
 [   0.     0.     0.    64.     0.     0. ]
 [   0.    80.    51.2    0.    80.     0. ]
 [  64.     0.     0.    64.     0.   100. ]
 [   0.    80.     0.     0.    80.   100. ]]
